# Google's Landmark Retrieval Challenge

**Goal**: Given over a million images of 15K classes, you need to classify them correctly as they lack labels. Such that given a test image, it should be able to retrieve images that belong to similar class.

[Kaggle Challenge](https://www.kaggle.com/c/landmark-retrieval-challenge)
[Amazing resource](https://blog.keras.io/building-autoencoders-in-keras.html)

In [ ]:
import pandas as pd
import urllib
import shutil
import cv2
import os
import numpy as np
from keras.preprocessing import image     

In [ ]:
#STEP #1: Download images
train_data = pd.read_csv("index.csv")
test_data = pd.read_csv("test.csv")

#print("Shape of training set = ", train_data.shape)
#print("Shape of testing set = ", test_data.shape)

def download_images(csv_file, outputpath, img_file_type):
    if not os.path.isdir(outputpath):
        os.makedirs(outputpath, exist_ok=True)
    
    count = 0
    for index, row in csv_file.iterrows():
        img_id = row['id']
        img_url = row['url']
        img_title = outputpath + '/' +img_id + '.' + img_file_type
        if index < 20:
            try:
                urllib.request.urlretrieve(img_url,img_title) # saves local copy of images
                count += 1
            except:
                print(img_id + "<-- couldn't retrieve IMAGE for this id!!")
        else:
            print("Downloaded ==> " + str(count) + " files!") 
            return

In [ ]:
#download_images( train_data, 'train','jpg')
#download_images(test_data, 'test','jpg')

In [ ]:
def load_dataset(inputpath):
    list_of_tensors = []
    for dirpath, dirnames, filenames in os.walk(inputpath):
        #print("Files in this '" + inputpath + "' dir is " + str(len(filenames)))
        for filename in filenames:
            img_path = os.path.join(dirpath,filename)
            img = image.load_img(img_path, target_size=(224,224))#,grayscale=True) #rescaling images to 224x224
            x = image.img_to_array(img)
            list_of_tensors.append(np.expand_dims(x, axis=0))
    return np.vstack(list_of_tensors)

In [ ]:
from keras.layers import Input, Dense
from keras.models import Model
from keras import regularizers

# this is the size of our encoded representations
encoding_dim = 6000  # 6000 floats -> compression of factor 25.088, assuming the input is 150528 floats

# this is our input placeholder
input_img = Input(shape=(150528,))
encoded = Dense(encoding_dim, activation='relu')(input_img)
decoded = Dense(150528, activation='sigmoid')(encoded)

# this model maps an input to its reconstruction
autoencoder = Model(input_img, decoded)

In [ ]:
# this model maps an input to its encoded representation
encoder = Model(input_img, encoded)

# # create a placeholder for an encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim,))
# # retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# # create the decoder model
decoder = Model(encoded_input, decoder_layer(encoded_input))

In [ ]:
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

In [ ]:
x_train = load_dataset('train')
x_test = load_dataset('test')

#We will normalize all values between 0 and 1 and we will flatten the 224x224x3 images into vectors of size 150528.
x_train_f = x_train.astype('float32')/255 
x_test_f = x_test.astype('float32')/255
# Convert 4D (numOfRows, 224, 224, 3) ==> (224x224x3)
x_train_f = x_train_f.reshape((len(x_train_f), np.prod(x_train_f.shape[1:])))
x_test_f = x_test_f.reshape((len(x_test_f), np.prod(x_test_f.shape[1:])))

print("Training set: ", x_train_f.shape)
print("Testing set: ", x_test_f.shape)

In [ ]:
import tensorflow as tf
init = tf.global_variables_initializer()
session = tf.Session()
session.run(init)

In [ ]:
autoencoder.fit(x_train_f, x_train_f, epochs=10, batch_size=500,  shuffle=True, validation_data=(x_test_f, x_test_f))

In [ ]:
# encode and decode some images
# note that we take them from the *test* set
encoded_imgs = encoder.predict(x_test_f)
decoded_imgs = decoder.predict(encoded_imgs)
#print(encoded_imgs.shape)

decoded_imgs = autoencoder.predict(x_test_f)
#print(decoded_imgs.shape)
#print(x_test_f.shape)

In [ ]:
import matplotlib.pyplot as plt

n = 5  # how many digits we will display
plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test_f[i].reshape(128, 128,3))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(128, 128,3))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()